<a href="https://colab.research.google.com/github/JacopoMangiavacchi/FastLabeling/blob/master/mnist_odd_even.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'fastai2>=0.0.11'

In [ ]:
from fastai2.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
path.ls()

(#4) [Path('oe_testing'),Path('testing'),Path('training'),Path('oe_training')]

In [ ]:
(path/'training').ls()

(#10) [Path('training/0'),Path('training/2'),Path('training/5'),Path('training/3'),Path('training/8'),Path('training/1'),Path('training/4'),Path('training/7'),Path('training/6'),Path('training/9')]

In [ ]:
(path/'oe_training').mkdir(exist_ok=True)
(path/'oe_training/odd').mkdir(exist_ok=True)
(path/'oe_training/even').mkdir(exist_ok=True)
(path/'oe_testing').mkdir(exist_ok=True)
(path/'oe_testing/odd').mkdir(exist_ok=True)
(path/'oe_testing/even').mkdir(exist_ok=True)
path.ls()

(#4) [Path('oe_testing'),Path('testing'),Path('training'),Path('oe_training')]

In [ ]:
from shutil import copy2

def copy_files(path, source_folder, sub_folders_list, destination_folder):
  for i in sub_folders_list:
    folder = source_folder + '/' + str(i)
    for file in (path/folder).iterdir():
      copy2(file, (path/destination_folder))

In [ ]:
copy_files(path, 'training', [0, 2, 4, 6, 8], 'oe_training/odd')
copy_files(path, 'training', [1, 3, 5, 7, 9], 'oe_training/even')
copy_files(path, 'testing', [0, 2, 4, 6, 8], 'oe_testing/odd')
copy_files(path, 'testing', [1, 3, 5, 7, 9], 'oe_testing/even')


In [ ]:
(path/'oe_training/odd').ls(), (path/'oe_training/even').ls(), (path/'oe_testing/odd').ls(), (path/'oe_testing/even').ls()

((#29492) [Path('oe_training/odd/43788.png'),Path('oe_training/odd/41885.png'),Path('oe_training/odd/5103.png'),Path('oe_training/odd/51611.png'),Path('oe_training/odd/35872.png'),Path('oe_training/odd/37302.png'),Path('oe_training/odd/12682.png'),Path('oe_training/odd/17299.png'),Path('oe_training/odd/36624.png'),Path('oe_training/odd/27618.png')...],
 (#30508) [Path('oe_training/even/22678.png'),Path('oe_training/even/23570.png'),Path('oe_training/even/36152.png'),Path('oe_training/even/47609.png'),Path('oe_training/even/18364.png'),Path('oe_training/even/54898.png'),Path('oe_training/even/1968.png'),Path('oe_training/even/57732.png'),Path('oe_training/even/58641.png'),Path('oe_training/even/9173.png')...],
 (#4926) [Path('oe_testing/odd/5103.png'),Path('oe_testing/odd/1968.png'),Path('oe_testing/odd/9173.png'),Path('oe_testing/odd/5806.png'),Path('oe_testing/odd/6097.png'),Path('oe_testing/odd/7423.png'),Path('oe_testing/odd/7706.png'),Path('oe_testing/odd/2060.png'),Path('oe_testin

In [ ]:
dls = ImageDataLoaders.from_folder(path, train='oe_training', valid='oe_testing')

dls.train_ds, dls.valid_ds

((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size=28x28, TensorCategory(1)),(PILImage mode=RGB size

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(5)


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.291701,0.216060,0.915100,01:17


epoch,train_loss,valid_loss,accuracy,time
0,0.061762,0.033810,0.987800,01:24
1,0.030088,0.019947,0.993400,01:23
2,0.011633,0.014930,0.994800,01:22
3,0.006577,0.015406,0.994700,01:23
4,0.001833,0.015210,0.995700,01:21


In [ ]:
test_path = (path/'testing'/'5').ls()[10]
test_image = Image.open(test_path)
test_image

In [ ]:
learn.predict(test_path)

('even', tensor(0), tensor([1.0000e+00, 2.0340e-07]))

In [ ]:
learn.validate()

In [ ]:
learn.summary()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)
interp.plot_top_losses(9, figsize=(7,7))